In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
import time, datetime
import pickle 

# Preprocessing data for building the ML model

### Loading the data

In [2]:
og_train_df = pd.read_csv('og_data/train.csv')
og_test_df = pd.read_csv('og_data/test.csv')
og_oil_df = pd.read_csv('og_data/oil.csv')
og_stores_df = pd.read_csv('og_data/stores.csv')
og_transactions_df = pd.read_csv('og_data/transactions.csv')
og_holidays_df = pd.read_csv('og_data/holidays_events.csv')

In [3]:
train_df = deepcopy(og_train_df)
test_df = deepcopy(og_test_df)
oil_df = deepcopy(og_oil_df)
stores_df = deepcopy(og_stores_df)
transactions_df = deepcopy(og_transactions_df) 

### Summary of original data

In [4]:
og_train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
og_oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
og_stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
og_transactions_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [33]:
oil_prices = np.asarray(og_oil_df.dcoilwtico)

### Remove the nan values from oil prices

In [34]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [35]:
oil_prices

array([  nan, 93.14, 92.97, ..., 46.46, 45.96, 47.26])

In [36]:
nans, x= nan_helper(oil_prices)
oil_prices[nans]= np.interp(x(nans), x(~nans), oil_prices[~nans])

In [38]:
oil_df['dcoilwtico'] = oil_prices

### Building the date conversion into unix time for ML model

In [48]:
date_min, date_max = oil_df.date.min(), oil_df.date.max()

In [49]:
date_min = time.mktime(datetime.datetime.strptime(date_min, "%Y-%m-%d").timetuple())
date_max = time.mktime(datetime.datetime.strptime(date_max, "%Y-%m-%d").timetuple())

In [42]:
og_test_df 

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [52]:
def GiveUnixTimeListNormalized(vector, date_format="%Y-%m-%d"):
    new_dates = np.zeros(len(vector), dtype=float)
    for i, s in enumerate(vector):
        new_dates[i] = time.mktime(datetime.datetime.strptime(s, date_format).timetuple())
    new_dates -= date_min
    new_dates /= date_max
    return new_dates

In [53]:
new_dates = GiveUnixTimeListNormalized(og_oil_df['date'], date_format="%Y-%m-%d")
oil_df['date'] = new_dates

In [55]:
new_dates = GiveUnixTimeListNormalized(og_train_df['date'], date_format="%Y-%m-%d")
train_df['date'] = new_dates

In [58]:
new_dates = GiveUnixTimeListNormalized(og_test_df['date'], date_format="%Y-%m-%d")
test_df['date'] = new_dates

In [64]:
train_df

,id,date,store_nbr,family,sales,onpromotion
0,0,0.000000,1,AUTOMOTIVE,0.000,0
1,1,0.000000,1,BABY CARE,0.000,0
2,2,0.000000,1,BEAUTY,0.000,0
3,3,0.000000,1,BEVERAGES,0.000,0
4,4,0.000000,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,0.096909,9,POULTRY,438.133,0
3000884,3000884,0.096909,9,PREPARED FOODS,154.553,1
3000885,3000885,0.096909,9,PRODUCE,2419.729,148
3000886,3000886,0.096909,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [63]:
oil_df

,date,dcoilwtico
0,0.000000,93.14
1,0.000057,93.14
2,0.000115,92.97
3,0.000172,93.12
4,0.000345,93.20
...,...,...
1213,0.097483,47.65
1214,0.097655,46.40
1215,0.097713,46.46
1216,0.097770,45.96


### Building the processed dictionaries to train and test the ML model

In [72]:
stores_transactions_dict = {}
for store in train_df['store_nbr'].unique():
    print(store)
    stores_transactions_dict[store] = {}
    for fam in train_df['family'].unique():
        stores_transactions_dict[store][fam] = {}
        stores_transactions_dict[store][fam]['date'] = train_df.where((train_df['store_nbr'] == store) & (train_df['family'] == fam))['date'].dropna().values.tolist()
        stores_transactions_dict[store][fam]['sales'] = train_df.where((train_df['store_nbr'] == store) & (train_df['family'] == fam))['sales'].dropna().values.tolist()
        stores_transactions_dict[store][fam]['onpromotion'] = train_df.where((train_df['store_nbr'] == store) & (train_df['family'] == fam))['onpromotion'].dropna().values.tolist()
        oil_list = []
        for date in stores_transactions_dict[store][fam]['date']:
            if date in oil_df['date']:
                oil_list.append(oil_df.where(date == oil_df['date'])['dcoilwtico'].values)
        stores_transactions_dict[store][fam]['oil_prices'] = oil_list

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
48
49
5
50
51
52
53
54
6
7
8
9


In [74]:
with open('Data/train_dictionary.pkl', 'wb') as f:
    pickle.dump(stores_transactions_dict, f)

In [78]:
stores_transactions_test_dict = {}
for store in test_df['store_nbr'].unique():
    print(store)
    stores_transactions_test_dict[store] = {}
    for fam in test_df['family'].unique():
        stores_transactions_test_dict[store][fam] = {}
        stores_transactions_test_dict[store][fam]['id'] = test_df.where((test_df['store_nbr'] == store) & (test_df['family'] == fam))['id'].dropna().values.tolist()
        stores_transactions_test_dict[store][fam]['date'] = test_df.where((test_df['store_nbr'] == store) & (test_df['family'] == fam))['date'].dropna().values.tolist()
        stores_transactions_test_dict[store][fam]['onpromotion'] = test_df.where((test_df['store_nbr'] == store) & (test_df['family'] == fam))['onpromotion'].dropna().values.tolist()
        oil_list = []
        for date in stores_transactions_test_dict[store][fam]['date']:
            if date in oil_df['date']:
                oil_list.append(oil_df.where(date == oil_df['date'])['dcoilwtico'].values)
        stores_transactions_test_dict[store][fam]['oil_prices'] = oil_list

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
48
49
5
50
51
52
53
54
6
7
8
9


In [83]:
with open('Data/test_dictionary.pkl', 'wb') as f:
    pickle.dump(stores_transactions_test_dict, f)

# From here we work with the dictionary processed data

In [85]:
with open('Data/train_dictionary.pkl', 'rb') as f:
    stores_transactions_dict = pickle.load(f)

with open('Data/test_dictionary.pkl', 'rb') as f:
    stores_transactions_test_dict = pickle.load(f)

In [84]:
for key in stores_transactions_dict.keys():
    print(key)

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
48
49
5
50
51
52
53
54
6
7
8
9
